In [1]:
import sqlite3
import pandas as pd
import numpy as np
import pyproj as _proj
from  os.path import join

## Collect the DRT results for simulation

In [3]:
fldr = '/media/pedro/storage/GDrive/DATA/Pedro/Professional/VLC/Projects/Vancouver/'

In [4]:
results = sqlite3.connect(join(fldr, 'SIMULATIONS/trunk and feeder/bus_passengers_3-0-10.sqlite'))
pax_qry = 'select * from passengers'
traces_qry = 'select * from vehicle_traces'
events_qry = 'select * from events'

### Vehicle traces

In [5]:
%%time
traces = pd.read_sql(traces_qry, results)

CPU times: user 1.94 s, sys: 132 ms, total: 2.07 s
Wall time: 2.07 s


In [414]:
nodelyr = sqlite3.connect(join(fldr, "Network/final_layer.gpkg"))
nds = pd.read_sql_query("SELECT fid, X, Y FROM nodes where pt_only=0", nodelyr)

# NEED TO TRANSFORM THE COORDINATES TO 385
# Already was in this case
source_crs = _proj.Proj(init='EPSG:4326')
cityphi_crs = _proj.Proj(init='EPSG:3857')


In [415]:
x = np.zeros(nds.fid.max()+1)
x[nds.fid] = nds.X
y = np.zeros(nds.fid.max()+1)
y[nds.fid] = nds.Y

nds = nds.assign(Z=0)
# Transformation would come here
# x, y = _proj.transform(source_crs, cityphi_crs, x, y)

vu = traces.vehicle_id.unique()
veh_ids = pd.DataFrame({'vehicle_id': vu, 'trip_id': np.arange(len(vu))+100000})

In [416]:
%%time
traces = traces.assign(x=x[traces.node_id])
traces = traces.assign(y=y[traces.node_id])
traces = traces.assign(minute=traces.instant / 60)
traces = traces.assign(z=0)

# Actual vehicle IDs
traces = traces.merge(veh_ids, on='vehicle_id')
# traces = traces.assign(trip_id=np.arange(traces.shape[0]) + 1)

Wall time: 2.72 s


Make all vehicles static after last dropoff

In [417]:
stay = traces.groupby('vehicle_id').last().reset_index()
stay.minute = 24 * 60
stay.task = 5
traces = pd.concat([traces, stay])

In [151]:
# traces.head()

People waiting to be picked up

In [418]:
%%time
passengers = pd.read_sql(pax_qry, results)

Wall time: 750 ms


In [419]:
%%time
passengers = passengers.assign(appeared=passengers.required_pickup / 60)
passengers = passengers.assign(serviced=passengers.actual_pickup / 60)
passengers = passengers.assign(pax_id=np.arange(passengers.shape[0]) + 1)
passengers = passengers.assign(from_x=x[passengers.origin])
passengers = passengers.assign(from_y=y[passengers.origin])
passengers = passengers.assign(z=0)


Wall time: 141 ms


In [420]:
passengers.head()

,id,origin,destination,group_size,required_pickup,actual_pickup,expected_travel_time,actual_travel_time,ideal_arrival_time,actual_arrival_time,extra_stops,appeared,serviced,pax_id,from_x,from_y,z
0,8c02399d-2290-4f85-a9a4-40a87f1e,8046,3119,1,5509.448,5509.448242,132.890790,192.891006,5642.338790,5702.339249,0,91.824133,91.824137,1,-1.366346e+07,6.293200e+06,0
1,0873b31c-4051-4a7a-a0c5-740842e7,1801,7832,1,5473.007,5473.006836,239.845638,299.845993,5712.852638,5772.852829,0,91.216783,91.216781,2,-1.364363e+07,6.308131e+06,0
2,33c0655c-c8c1-4ee1-a8da-1c9da773,1951,7284,1,5419.766,5503.528802,281.942904,341.942993,5701.708904,5845.471795,0,90.329433,91.725480,3,-1.366893e+07,6.303773e+06,0
3,e06d0343-32be-4526-a558-ece9d16a,6792,4828,1,5566.570,5582.417470,225.691170,285.690994,5792.261170,5868.108464,0,92.776167,93.040291,4,-1.370390e+07,6.323098e+06,0
4,aa2c3e96-74bc-4381-a294-44fa165e,4078,4098,1,5528.253,5528.252930,301.732632,361.733002,5829.985632,5889.985931,0,92.137550,92.137549,5,-1.370687e+07,6.299427e+06,0


In [421]:
results.close()

## Do we need to make any transformations on the data?

In [ ]:
# Maybe

# We start the CityPhi work

### Start CityPhi
* Cannot be done over RDP with GTX or RTX GPUs

In [1]:
import cityphi.application
%gui cityphi
app = cityphi.application.Application()
import cityphi.feature as _feat

In [364]:
import cityphi.widget as _widget
import cityphi.attribute as _att
import cityphi.layer as _layer
import cityphi.parameter as _param

### Builds the trace points  layer

In [437]:
tr = traces.sort_values(by=['trip_id', 'minute'])
vehicle_trip_ids = tr.trip_id.values
vehicle_points = tr[["x", "y", "z"]].values
vehicle_times = tr.minute.values
occupancy = tr.occupancy.values.astype(np.float32) + 1
occupancy = occupancy * occupancy

In [438]:
vehicle_feature = _feat.TrajectoryFeature(vehicle_trip_ids, vehicle_points, vehicle_times)

In [439]:
# vehicle_feature.add_time_varying_attribute("occupancy", "float32", vehicle_trip_ids,vehicle_times, occupancy)
vehicle_feature.add_attribute("occupancy", "float32", vehicle_trip_ids, occupancy)

In [440]:
class OccupancyAttribute(_att.Attribute):
    name = 'Occupancy' # this is the name displayed in the UI
    time = _param.FloatParameter()   
    def __init__(self):
        _att.Attribute.__init__(self)
        self.time = 0

    def load(self, feature_data):
        return feature_data['occupancy', self.time]

# instantiate the custom attribute and set vehicle size
vehicle_layer.radius = OccupancyAttribute()


### Create animated trajectory layers

In [453]:
vehicle_layer = _layer.AnimatedPointLayer(vehicle_feature, max_vertices = 2000000) # max_verticves is performance tunning
vehicle_layer.name = "DRT"
app.add_layer(vehicle_layer)

In [454]:
vehicle_layer.glowing = False
vehicle_layer.min_pixel_size = 4
vehicle_layer.color = (0, 0, 255) # A relatively bright yellow
app.background_color = (25, 25, 25)  # make the background black / dark gray

In [455]:
motion_trail_layer = _layer.TrajectoryLayer(vehicle_feature, max_vertices = 2000000)
motion_trail_layer.name = "Motion trails"
app.add_layer(motion_trail_layer)
motion_trail_layer.color = vehicle_layer.color
motion_trail_layer.width = vehicle_layer.radius
motion_trail_layer.priority = 6

### Create Node layer

In [447]:
node_ids = nds.fid.values
points = nds[["X", "Y", "Z"]].values

nodes_feat = _feat.PointFeature(node_ids, points)
node_layer = _layer.PointLayer(nodes_feat)
node_layer.name = "Network Nodes"
# app.add_layer(node_layer)

### Create layer for pickups

In [448]:
pax_ids = passengers.pax_id.values
node_ids = passengers.origin.values
pickup_feat = _feat.PointFeature.from_points(pax_ids, node_ids, nodes_feat)

In [449]:
pickup_feat.add_attribute("start_time", "int32", pax_ids, passengers.appeared.values)
pickup_feat.add_attribute("end_time", "int32", pax_ids, passengers.serviced.values)

In [450]:
pickup_layer = _layer.PointLayer(pickup_feat)
pickup_layer.name = "Passenger Pickups"
pickup_layer.height = 20
pickup_layer.min_pixel_size = 4
pickup_layer.stacked = True

pickup_layer.start_time = _att.FeatureAttribute("start_time")
pickup_layer.end_time = _att.FeatureAttribute("end_time")
app.add_layer(pickup_layer)

Add a time slider

In [456]:
# Set up time slider
import datetime

# Current activities
def change_time(t):
    vehicle_layer.time = t
    vehicle_layer.radius.time = t
    pickup_layer.time_window = t, t + 2
    motion_trail_layer.time_window = (t - 0.1, t)
    
def format_time(t):
    return str(datetime.timedelta(minutes=t))
#     return "%02d:%02d" % (t / 60, t % 60)
    
time_slider = _widget.TimeSlider(90, 1440, format_time, change_time)
app.add_widget(time_slider)
time_slider.pause()

In [82]:
q = traces[traces.trip_id == 100028]
q.to_csv(join(fldr, 'q.csv'), index=False)

In [ ]:
# Jump to 7:15 am to watch the vehicle animation
# Zoom out to see the full trajectories
time_slider.time = 435
time_slider.play()
app.camera.set_view({'distance': 3398.9692211802712,
                     'rotation': (359.0, 30.0),
                     'translation': (-13618189.092867726, 6042003.390481629)})
